In [7]:
import transformers
import torch
from transformers import (
    BertModel, 
    BertTokenizer, 
    AutoModelForMaskedLM, 
    DataCollatorForLanguageModeling,
    Trainer
)
import datasets
from datasets import load_dataset, load_metric
import pandas as pd
import os
import numpy as np
from torch.utils.data import DataLoader
import tqdm

In [15]:
line = "ASDFASDF"

In [20]:
df = pd.read_csv("heavy_ighm/Heavy_IGHM.csv")

In [21]:
df.head()

,sequence,locus,stop_codon,vj_in_frame,v_frameshift,productive,rev_comp,complete_vdj,v_call,d_call,...,cdr3_start,cdr3_end,np1,np1_length,np2,np2_length,c_region,Redundancy,ANARCI_numbering,ANARCI_status
0,CGGTCCCTGAGACTCTCCTGTGCAGCCTCTGGATTCACCTTTGATG...,H,F,T,F,T,T,F,IGHV3-43*02,IGHD3-22*01,...,244.0,297.0,TAACTTCCAGT,11.0,CAT,3.0,GGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGAAGC,18,"{'fwh1': {'18 ': 'S', '19 ': 'L', '20 ': 'R', ...","|Deletions: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,..."
1,GGTCCCTGAGACTCTCCTGTGCAGCGTCTGGATTCATCTTTAGTAG...,H,F,T,F,T,T,F,IGHV3-7*01,IGHD1-26*01,...,243.0,272.0,CCCTC,5.0,GTTT,4.0,GGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGG,35,"{'fwh1': {'18 ': 'S', '19 ': 'L', '20 ': 'R', ...","|Deletions: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,..."
2,GCAGCTGGTGCAGTCTGGAGCTGAAGTGAAGAAGCCTGGGGCCTCA...,H,F,T,F,T,T,F,IGHV1-18*01,IGHD1-26*01,...,284.0,325.0,GGGGAAGTG,9.0,AGG,3.0,GGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGAACG,44,"{'fwh1': {'3 ': 'Q', '4 ': 'L', '5 ': 'V', '6 ...","|Deletions: 1, 2, 10, 73|||Shorter than IMGT d..."
3,TACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATCAGCAGT...,H,F,T,F,T,T,F,IGHV4-39*07,IGHD4-23*01,...,245.0,286.0,AGCC,4.0,TCTT,4.0,GGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGTC,1,"{'fwh1': {'18 ': 'T', '19 ': 'L', '20 ': 'S', ...","|Deletions: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,..."
4,ACCCTGTCCCTCACCTGCACTGTCTCTGGTGGCTCCATCAGCAGTA...,H,F,T,F,T,T,F,IGHV4-39*01,IGHD3-22*01,...,244.0,288.0,TCCC,4.0,C,1.0,GGAGTGCATCCGCCCCAACCCTTTTCCCCCTCGTCTCCTGACTT,47,"{'fwh1': {'18 ': 'T', '19 ': 'L', '20 ': 'S', ...","|Deletions: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,..."


In [22]:
df.columns

Index(['sequence', 'locus', 'stop_codon', 'vj_in_frame', 'v_frameshift',
       'productive', 'rev_comp', 'complete_vdj', 'v_call', 'd_call', 'j_call',
       'sequence_alignment', 'germline_alignment', 'sequence_alignment_aa',
       'germline_alignment_aa', 'v_alignment_start', 'v_alignment_end',
       'd_alignment_start', 'd_alignment_end', 'j_alignment_start',
       'j_alignment_end', 'v_sequence_alignment', 'v_sequence_alignment_aa',
       'v_germline_alignment', 'v_germline_alignment_aa',
       'd_sequence_alignment', 'd_sequence_alignment_aa',
       'd_germline_alignment', 'd_germline_alignment_aa',
       'j_sequence_alignment', 'j_sequence_alignment_aa',
       'j_germline_alignment', 'j_germline_alignment_aa', 'fwr1', 'fwr1_aa',
       'cdr1', 'cdr1_aa', 'fwr2', 'fwr2_aa', 'cdr2', 'cdr2_aa', 'fwr3',
       'fwr3_aa', 'fwr4', 'fwr4_aa', 'cdr3', 'cdr3_aa', 'junction',
       'junction_length', 'junction_aa', 'junction_aa_length', 'v_score',
       'd_score', 'j_score', 'v_

In [23]:
df[[x for x in df.columns if 'cdr' in x]]

,cdr1,cdr1_aa,cdr2,cdr2_aa,cdr3,cdr3_aa,cdr1_start,cdr1_end,cdr2_start,cdr2_end,cdr3_start,cdr3_end
0,GGATTCACCTTTGATGATTATGCC,GFTFDDYA,ATTAGTGGGGATGGTGGTAGCACA,ISGDGGST,GCAAAAGATATAACTTCCAGTTATTACTATGATAGTAGTGGTCATT...,AKDITSSYYYDSSGHFDY,31.0,54.0,106.0,129.0,244.0,297.0
1,GGATTCATCTTTAGTAGCTATTGG,GFIFSSYW,ATAAAGCAAGATGGAAGTGAGAAA,IKQDGSEK,GCGAGAGCCCTCTATAGTGGGAGGTTTTAC,ARALYSGRFY,30.0,53.0,105.0,128.0,243.0,272.0
2,GGTTACACCTTTACCAGCTATGGT,GYTFTSYG,ATCAGCGCTTACAATGGTAACACA,ISAYNGNT,GCGAGAGGGGGAAGTGTAGTGGGATCTACTACAGGTGACTAC,ARGGSVVGSTTGDY,71.0,94.0,146.0,169.0,284.0,325.0
3,GGTGGCTCCATCAGCAGTAGTAGTTACTAC,GGSISSSSYY,ATCTATTATAGTGGGAGCACC,IYYSGST,GCGAAGCCCTACAGTGGTAACTTCTTTGATGCTTTTGATATC,AKPYSGNFFDAFDI,29.0,58.0,110.0,130.0,245.0,286.0
4,GGTGGCTCCATCAGCAGTAGTAGTTACTAC,GGSISSSSYY,ATCTATTATAGTGGGAGCACC,IYYSGST,GCGAGACATCCCTATTACTATGATAGTAGTGGTTCCTTTGACTAC,ARHPYYYDSSGSFDY,28.0,57.0,109.0,129.0,244.0,288.0
...,...,...,...,...,...,...,...,...,...,...,...,...
435282,GGTGGCTCCATCAGTACTAACTAC,GGSISTNY,ATCCATTACACTGGGAGCACC,IHYTGST,GCGAGAGGGGCGAGAGCCATAGCACCTCGCCTCTTTGACTCC,ARGARAIAPRLFDS,52.0,75.0,127.0,147.0,262.0,303.0
435283,GGATTCATGTTTAGCAGCTATGCC,GFMFSSYA,ATTAGTGGTAGTGGTGGGAGCACA,ISGSGGST,GCGAGGGGCGATACAGCAGTGGCTGGGGAGGATGGTTTTGATATG,ARGDTAVAGEDGFDM,32.0,55.0,107.0,130.0,245.0,289.0
435284,GGTGGCTCCATCACTAATTACTAC,GGSITNYY,ATCTATTACAGTGGGAGCACC,IYYSGST,GCGAGAGAAGCCGGAGTTAGGGCTCCCCTTGACGAC,AREAGVRAPLDD,30.0,53.0,105.0,125.0,240.0,275.0
435285,GGTGACTCCATCAGCAGTCGTAGTTACTAC,GDSISSRSYY,ATCTATACCAGTGGGAGCACC,IYTSGST,GCGAGAGTGTATAGCAGTGGCTGGTATAATGCTTTTGATATC,ARVYSSGWYNAFDI,51.0,80.0,132.0,152.0,267.0,308.0


# Train LM

In [ ]:
model_name = "Rostlab/prot_bert_bfd"
dataset_name = "sequences"
cache_dir = "./cache"
validation_split_percentage = 5

In [ ]:
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)

In [101]:
raw_datasets = load_dataset(
    "text",
    data_files=data_files,
    cache_dir=cache_dir
)

Using custom data configuration default-0ef9a2c30a4c85ff
Reusing dataset text (./cache/text/default-0ef9a2c30a4c85ff/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


  0%|          | 0/1 [00:00<?, ?it/s]

In [102]:
if "validation" not in raw_datasets.keys():
    raw_datasets["validation"] = load_dataset(
        "text",
        data_files=data_files,
        split=f"train[:{validation_split_percentage}%]",
        cache_dir=cache_dir
    )
    raw_datasets["train"] = load_dataset(
        "text",
        data_files=data_files,
        split=f"train[{validation_split_percentage}%:]",
        cache_dir=cache_dir
    )

Using custom data configuration default-0ef9a2c30a4c85ff
Reusing dataset text (./cache/text/default-0ef9a2c30a4c85ff/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)
Using custom data configuration default-0ef9a2c30a4c85ff
Reusing dataset text (./cache/text/default-0ef9a2c30a4c85ff/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8)


In [103]:
data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm_probability=0.15,
        pad_to_multiple_of=8
)

In [104]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # preds have the same shape as the labels, after the argmax(-1) has been calculated
    # by preprocess_logits_for_metrics
    labels = labels.reshape(-1)
    preds = preds.reshape(-1)
    mask = labels != -100
    labels = labels[mask]
    preds = preds[mask]
    return metric.compute(predictions=preds, references=labels)

In [105]:
def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        # Depending on the model and config, logits may contain extra tensors,
        # like past_key_values, but logits always come first
        logits = logits[0]
    return logits.argmax(dim=-1)

In [106]:
def tokenize_function(examples):
    # Remove empty lines
    examples["text"] = [
        line for line in examples["text"] if len(line) > 0 and not line.isspace()
    ]
    
    return tokenizer(
        examples["text"],
        padding="max_length",
        truncation=True,
        max_length=128,
        # We use this option because DataCollatorForLanguageModeling (see below) is more efficient when it
        # receives the `special_tokens_mask`.
        return_special_tokens_mask=True,
    )

In [107]:
tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    desc="Running tokenizer on dataset line_by_line",
    load_from_cache_file=True,
    remove_columns=["text"]
)

Loading cached processed dataset at ./cache/text/default-0ef9a2c30a4c85ff/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-7f96015c4986f7cd.arrow
Loading cached processed dataset at ./cache/text/default-0ef9a2c30a4c85ff/0.0.0/4b86d314f7236db91f0a0f5cda32d4375445e64c5eda2692655dd99c2dac68e8/cache-f5081781e74c7a2b.arrow


In [108]:
train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["validation"]

In [120]:
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30, 1024, padding_idx=0)
      (position_embeddings): Embedding(40000, 1024)
      (token_type_embeddings): Embedding(2, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwi

In [133]:
train_data_loader = DataLoader(train_dataset, collate_fn=data_collator, batch_size=8)

In [136]:
for i, batch in enumerate(train_data_loader):
    print(i)
    model(batch['input_ids'])

0
1


KeyboardInterrupt: 

In [109]:
trainer = Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [110]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMaskedLM.forward` and have been ignored: special_tokens_mask. If special_tokens_mask are not expected by `BertForMaskedLM.forward`,  you can safely ignore this message.
/Users/ido/.pyenv/versions/miniforge3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 166962
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 62613


Step,Training Loss


KeyboardInterrupt: 